In [1]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [2]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO
plotly()

Plots.PlotlyBackend()

### Constantes

In [3]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

### Modele pour une microlentille

In [4]:
#bbox = BoundingBox(xmin=1, ymin=1, xmax=15, ymax=50);
bbox = BoundingBox(xmin=1, ymin=1, xmax=7, ymax=45); #size of bbox
#a0= 1. .- 0.2 .* randn(Float64,length(λlaser));
a0 = [300.0 , 400.0, 500.0];
#fwhm0= 5. .-  2. .* rand(Float64,length(λlaser));
fwhm0 = [2.0, 2.0 , 2.0];
laser =  LaserModel(λlaser,a0,fwhm0);
lmod = LensletModel(λ0,laser.nλ-1,bbox);

C0 = zeros(2,laser.nλ)
#C0[1:2,1:2] = [[ 6.2 10]; [25 80]]
C0[1:2,1:3] = [[ 4.0 0.0 0.0]; [22.0 -100.0 0.0]]
UpdateDispModel(lmod.dmodel,C0)
lmod.dmodel.(λlaser)
limage = LensletLaserImage(lmod,laser)

#Loi de dispersion
#wxy0 = lmod.dmodel.(wavelengthrange)
#(wx0,wy0) = collect.( zip(wxy0...))

#affichage de l'image
heatmap(limage, title = "Donnees simulees")
#plot!(wy0,wx0; label="dispersion law")

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
#data= limage .+ 0.05 .*randn(Float64,size(limage)) ;  #ajout de bruit 
#heatmap(data; title="Modèle simulé avec paramètres connus + bruit")

## Donnees reelles:

In [6]:
f3 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")
data= read(f3[1]);

#Bounding Box de la lenslet
cx = -10 + 1025;
cy = 45 + 1025;
radx = 4;
rady = 22;

ldata = data[1012:1018 , 1048:1092 ]
heatmap(ldata, title = "Donnees reelles")

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
#Tableau
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((ldata .- LensletLaserImage(lmod,laser)).^2)
end

@show cost = likelihood(a0,fwhm0,C0);
∇cost = gradient(likelihood,a0,fwhm0,C0);

cost = likelihood(a0, fwhm0, C0) = 2.852052013806447e6


In [8]:
#Avec bruit
@show cost = likelihood(a0,fwhm0.+ 2.,C0)
∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,C0);

cost = likelihood(a0, fwhm0 .+ 2.0, C0) = 2.4383606080786767e6


In [9]:
#Vecteur: optimisation plus rapide car les trois coefficients a traiter sont donnes en meme temps
function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:(laser.nλ)],x[(laser.nλ+1):(2*laser.nλ)],reshape(x[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((ldata .- LensletLaserImage(lmod,laser)).^2)
end

likelihood (generic function with 2 methods)

In [10]:
x0 = vcat([a0[:],fwhm0[:],C0[:] ]...) #concatenation de a0, fwhm0 et C0

@show cost = likelihood(x0)
∇cost = gradient(likelihood,x0);

cost = likelihood(x0) = 2.852052013806447e6


### Optimisation

In [70]:
#Parametres d'initialisation
ainit = [300.0 , 400.0, 500.0];#a0  .+ (rand(Float64,laser.nλ) .- 0.5);
fwhminit = [2.0, 2.0 , 2.0]; #fwhm0  .+ (rand(Float64,laser.nλ) .- 2);
cinit = zeros(2,laser.nλ)
cinit[1:2,1:3] = [[ 4.0 0 0]; [21 -80 0]] 

UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);

#wxyinit = lmod.dmodel.(wavelengthrange)

#(wxinit,wyinit) = collect.( zip(wxyinit...))
xinit = vcat([ainit[:],fwhminit[:],cinit[:]]...)

∇cost = gradient(likelihood,xinit )

xopt = vmlmb(likelihood, xinit; verb=50, ftol=(0.0,0),gtol = (0.0,0));

# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    5.1320943055662578E+06  6.63E+05  0.00E+00
    50     65      0    2.6836431154378843E+06  1.36E+05  1.00E+00
   100    127      0    2.0430797030268768E+06  2.28E+05  1.00E+00
   150    182      0    1.8969530386732081E+06  8.81E+04  1.00E+00
   200    240      0    1.6282009537238136E+06  9.14E+03  1.00E+00
   250    294      0    1.5298005670323460E+06  1.39E+04  1.00E+00
   300    349      0    1.5269967740047621E+06  5.76E+02  1.00E+00
   350    402      0    1.5269856424128497E+06  1.68E+03  1.00E+00
   400    460      0    1.5268982929099326E+06  2.32E+02  1.00E+00
   450    515      0    1.5268918335744645E+06  5.12E+02  1.00E+00
   500    570      0    1.5268639330223685E+06  7.52E+02  1.00E+00
   550    623      0    1.5268430574660790E+06  1.50E+03  1.00E+00
   600    679      0    1.5265542169680858E+06  3.20E+02  1.00E+

### Comparaison

In [71]:
#Parametres d'optimisation
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);

In [72]:
@show(ainit,aopt);

@show(fwhminit,fwhmopt);

@show(cinit,copt);


#Donnees simulees
#heatmap(limage, title="Donnees simulees")


ainit = [300.0, 400.0, 500.0]
aopt = [957.6297839687048, 400.62381558386863, 64.22784451117751]
fwhminit = [2.0, 2.0, 2.0]
fwhmopt = [2.1836557307254205, 0.0031358875260142144, 39.11348073906345]
cinit = [4.0 0.0 0.0; 21.0 -80.0 0.0]
copt = [2.8101047188847366 6.744296319149254 5.123873994749534; 17.878876830708894 -69.65150227202648 -1.183979082406328]


In [73]:
#Donnee simulee bruitee
heatmap(ldata, title="Donnees reelles")

<!DOCTYPE html>
 
 
 Plots.jl

In [74]:
#Point de depart de l'optimiseur
UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmod,laser)

heatmap(limageinit, title = "Initialisation")

<!DOCTYPE html>
 
 
 Plots.jl

In [75]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmod,laser)

heatmap(limageopt, title = "Point d'arrivee de l'optimiseur")


<!DOCTYPE html>
 
 
 Plots.jl

In [77]:
#Residus
heatmap( ldata .- limageopt, title = "Residus")

<!DOCTYPE html>
 
 
 Plots.jl